<a href="https://colab.research.google.com/github/dhuMuhammadasif/test/blob/master/1_3_23_Presiach_Model_with_Gussian_distribution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# https://github.com/radiasoft/rsradia/blob/8cab493ce5640e60140f42269ee817f2c0a6a584/rsradia/hysteresis/preisach.py
"https://github.com/radiasoft/rsradia/blob/8cab493ce5640e60140f42269ee817f2c0a6a584/rsradia/hysteresis/preisach.py"

In [2]:
### distributions.py
### July 2022
### Distribution functions for probabilities defined over 2D grids

from numpy import array, zeros, exp, pi, einsum
from scipy.linalg import det, inv

# Note: in all functions below, the "grid" argument should be a numpy array with
# shape (n, 2), in which n is the total number of pixels (n = nx * ny) and the 
# 2 elements per pixel are its x/y values in the grid.

# Defines a discrete uniform pdf
def UNIFORM(grid):
    n_grid = array(grid.shape).prod()
    return zeros(n_grid)+1/n_grid

# Defines a discrete Gaussian pdf
def GAUSSIAN(grid, mu=array([0, 0]), Sigma=array([[.1, 0], [0, .1]])):
    
    # ARGS
    # mu: distribution mean (x/y coordinate pair)
    # Sigma: distribution covariance matrix
    
    # Determine distances between grid points & the mean
    dists = grid-mu

    # Looping over mixands, add to mixture pdf
    reg = ((2*pi)**2 * det(Sigma))**-.5
    pX = reg*exp(-0.5*einsum('ij,jl,il->i', dists, inv(Sigma), dists))

    return pX/pX.sum()

# Defines a distrete Gaussian mixture pdf
def GMIXTURE(grid, mus=array([[0, 0]]), Sigmas=array([[[.1, 0], [0, .1]]]), ws=[1]):
    
    # ARGS
    # mus: distribution means (x/y coordinate pairs) for each mixand
    # Sigmas: distribution covariance matrices for each mixand
    # ws: weights for each mixand

    # Define number of mixands & dimensions
    N, k = mus.shape

    # Initialize the mixture pdf
    pX = zeros(grid.shape[0])

    # Looping over mixands, add to mixture pdf
    for n in range(N):
        reg = ((2*pi)**k*det(Sigmas[n]))**-.5
        dists = (grid-mus[n])
        pX += ws[n]*reg*exp(-0.5*einsum('ij,jl,il->i', dists, inv(Sigmas[n]), dists))

    return pX/pX.sum()

In [3]:
PI = 3.141592653589793
MU0 = 4*PI*1e-7

In [7]:
print(MU0)

1.2566370614359173e-06


In [6]:
from numpy import array, zeros, ceil, argsort, ndarray, diff, sign
from pickle import dump, load
PI = 3.141592653589793
MU0 = 4*PI*1e-7
#from . import MU0
#from .distributions import *

# Define available options for probability density functions
DISTRIBUTIONS = {
    'UNIFORM': UNIFORM,
    'GAUSSIAN': GAUSSIAN,
    'GMIXTURE': GMIXTURE,
}

class Preisach:
    '''
    A Preisach model of magnetic hysteresis
    Parameters:
        Ms: Saturation magnetization of the magnet
        ab_max: Maximum magnitude of hysteron on/off field values
        ab_res: Resolution of alpha/beta grid (point spacing)
        distribution: Type of weight distribution used
        dist_params: Parameters of the weight distribution
    '''
    
    # Initializes a Preisach hysteresis model
    def __init__(self, Ms, ab_max, ab_res, dH=1, distribution="GAUSSIAN", dist_params=None):

        # Raise exceptions for invalid input parameters
        if Ms<=0: raise ValueError("\"Ms\" (saturation magnetization) must be a positive, non-zero number")
        if ab_max<=0: raise ValueError("\"ab_max\" (maximum hysteron grid value) must be a positive, non-zero number")
        if ab_res<=0: raise ValueError("\"ab_res\" (hysteron grid resolution) must be a positive, non-zero number")
        if dH<=0: raise ValueError("\"dH\" (field step-size) must be a positive, non-zero number")

        # Set valid input parameters
        self._Ms = Ms
        self._ab_max = ab_max
        self._ab_res = ab_res
        self._dH = dH
            
        # Construct the scaled Preisach grid
        n_grid = int(round(ceil(2*self._ab_max/self._ab_res)))
        self._grid = -1+2*array([[a,b] for a in range(n_grid+1) for b in range(n_grid+1)])/n_grid

        # Raise exceptions for invalid input distributions & parameters
        if isinstance(distribution, str):
            if distribution.upper() not in DISTRIBUTIONS.keys():
                raise ValueError("Invalid choice of distribution, must be one of: {:s}".format(', '.join(DISTRIBUTIONS.keys())))
            else:
                try:
                    self._density = DISTRIBUTIONS[distribution.upper()](self._grid, *dist_params)
                except:
                    raise RuntimeError("Unable to call {:s} with arguments in \"dist_params\"".format(distribution.upper()))
        elif isinstance(distribution, ndarray):
            if distribution.min()<0:
                raise ValueError("Custom distributions must contain only non-negative values")
            elif distribution.shape[0]!=self._grid.shape[0]:
                raise ValueError("Custom distributions must be the same shape as the Preisach grid")
            else:
                self._density = distribution
        else:
            raise ValueError("\"distribution\" must be a str or a numpy.ndarray")        

        # Reduce the Preisach density to its upper half & normalize, rescale the Preisach grid
        self._density[self._grid[:,0]>=self._grid[:,1]] = 0        
        self._density /= self._density.sum()
        self._grid *= self._ab_max
        
        # Compute the major hysteresis curve for the model (sets self.H_major, self.B_major)
        self._get_major()
        self._get_criticals()
            
    # Computes the initial & major 
    def _get_major(self):
        
        # Initialize the applied field & magnetization
        H = [0]
        M = [0]
        
        # Initialize hysteron values
        R = zeros(len(self._grid))
        R[-self._grid[:,0] >= self._grid[:,1]] = 1
        R[-self._grid[:,0] < self._grid[:,1]] = -1
        R[self._grid[:,0]>=self._grid[:,1]] = 0
        
        # Increase applied field until saturation is reached
        while (H[-1]<self._ab_max):
            H.append(H[-1] + self._dH)
            R[H[-1]>self._grid[:, 1]] = 1
            M.append(self._density@R)
            
        n_curve = int(round(ceil(2*max(H)/self._dH)))
        
        # Decrease applied field until negative saturation is reached
        for i in range(n_curve):
            H.append(H[-1] - self._dH)
            R[H[-1]<self._grid[:, 0]] = -1
            M.append(self._density@R)
            
        # Increase applied field until saturation is reached again
        for i in range(n_curve):
            H.append(H[-1] + self._dH)
            R[H[-1]>self._grid[:, 1]] = 1
            M.append(self._density@R)
              
        # Assign the hysteresis variables to model class members
        self.H_major = array(H)
        self.B_major = self._Ms*MU0*array(M)
        
    # Computes the material coercivity & remanent field
    def _get_criticals(self):        
        
        # Separate the major loop into upper & lower sections
        upper_inds = range(int(.2*len(self.H_major)), int(.6*len(self.H_major)))
        lower_inds = range(int(.6*len(self.H_major)), int(len(self.H_major)))
        H_upper = self.H_major[upper_inds]
        H_lower = self.H_major[lower_inds]
        B_upper = self.B_major[upper_inds]
        B_lower = self.B_major[lower_inds]
        
        # Determine the remanent field
        if 0 in H_upper:
            Bru = float(B_upper[H_upper==0])
        else:
            H0 = int(diff(sign(H_upper)).nonzero()[0])
            Bru = float(B_upper[H0:H0+2].sum()/2)
        if 0 in H_lower:
            Brl = float(B_lower[H_lower==0])
        else:
            H0 = int(diff(sign(H_lower)).nonzero()[0])
            Brl = float(B_lower[H0:H0+2].sum()/2)
        self.remanence = array([Brl, Bru])
            
        # Determine the coercivity
        if 0 in B_upper:
            Hcu = float(H_upper[B_upper==0])
        else:
            B0 = int(diff(sign(B_upper)).nonzero()[0])
            Hcu = float(H_upper[B0:B0+2].sum()/2)
        if 0 in B_lower:
            Hcl = float(H_lower[B_lower==0])
        else:
            B0 = int(diff(sign(B_lower)).nonzero()[0])
            Hcl = float(H_lower[B0:B0+2].sum()/2)
        self.coercivity = array([Hcu, Hcl])
        
    # Use the model to compute hysteresis curves along a path of applied fields
    def path(self, H_path, M0):
        
        # Determine numbers of hysteresis points & field change signs along path
        N_path = (ceil(H_path.ptp(axis=1))/self._dH).round().astype('int32')+1
        sgn_path = array([1 if Hs[1]>Hs[0] else -1 for Hs in H_path])
        
        # Initialize hysteresis variable containers
        H = zeros(N_path.sum())
        M = zeros(N_path.sum())
        
        # Initializehysteron values
        ab0 = (M0/self.B_major.max()/MU0)*self._ab_max
        R = zeros(len(self._grid))
        R[-self._grid[:,0] >= self._grid[:,1]-ab0] = 1
        R[-self._grid[:,0] < self._grid[:,1]-ab0] = -1
        
        # Loop over magnetizing field paths
        for p in range(len(H_path)):
            
            # Define starting & stopping field strengths for this path
            Hstart, Hstop = H_path[p]
            sgn = sgn_path[p]
            
            # Determine starting index & number of points on this path
            N0 = N_path[:p].sum()
            N = N_path[p]
            
            # Compute the hysteresis along this path
            for n in range(N):
                H[N0+n] = Hstart + n*self._dH*sgn
                if sgn>0: R[H[N0+n] > self._grid[:, 1]] = 1
                else: R[H[N0+n] < self._grid[:, 0]] = -1
                M[N0+n] = self._density@R
        
        return H, self._Ms*MU0*M
                
    # Use the model to compute magnetic flux density at a particular field strength
    def point(self, H_point, curve='upper'):
        
        # Indices for points along each curve
        curve_indices = {
            'initial': (0, int(.2*len(self.H_major))),
            'upper': (int(.2*len(self.H_major)), int(.6*len(self.H_major))),
            'lower': (int(.6*len(self.H_major)), int(len(self.H_major)))
        }
        
        # Select points along the desired curve
        if curve.lower() in curve_indices.keys():
            indices = curve_indices[curve.lower()]
        else:
            print("WARNING: Curve choice not understood, using upper curve.")
            indices = curve_indices['upper']
        H = self.H_major[indices[0]:indices[1]]
        B = self.B_major[indices[0]:indices[1]]
        
        # Find curve point closest to prescribed point
        H_dists = abs(H - H_point)
        neighbors = argsort(H_dists)[:2]
            
        # Interpolate magnetic flux density between neighboring points
        return sum([B[pt]*H_dists[pt] for pt in neighbors])/self._dH
    
    #
    @classmethod
    def fit(cls, **fit_params):
        """
        
        Parameters:
            
        """
        return 0
    
    # Saves a Preisach model to a file
    def save(self, path="./self.pkl"):
        with open(path,'wb') as file:
            dump(self, file)
    
    # Loads a Preisach model from a file (possibly on instantiation)
    @classmethod
    def load(cls, path="./self.pkl"):
        with open(path, 'rb') as file:
            return load(file)

In [ ]:
### integrators.py
### July 2022
### A collection of numerical integrators

from numpy import array, zeros, ceil

def KRON15(fun, bounds):
    """The 15-point Gauss-Kronrod quadrature integrator"""
    
    # Define quadrature points & weights on [-1,1] interval
    xs = [0.0, 0.207784955007898, 0.405845151377397, 0.586087235467691,\
         0.741531185599394, 0.864864423359769, 0.949107912342759, 0.991455371120813]
    ws = [0.209482141084728, 0.204432940075298, 0.190350578064785, 0.169004726639267,\
          0.140653259715525, 0.104790010322250, 0.063092092629979, 0.022935322010529]
    xs = array(xs[::-1][:-1] + xs)
    ws = array(ws[::-1][:-1] + ws)
    xs[:7] *= -1
    
    # Scale quadrature points & weights to desired interval
    xs = (bounds[1]-bounds[0])*(xs+1)/2+bounds[0]
    ws = (bounds[1]-bounds[0])*ws/2
    
    # Evaluate function at quadrature points & return weighted result
    return ws@fun(xs)

def EULER(X0, bounds, dt, diffEq, *diffArgs):
    """A simple Euler integrator"""
    
    isScalar = isinstance(X0,(int,float))
    N = int(round(ceil((bounds[1]-bounds[0]))/dt))+1
        
    # Intialize state scalar or vector
    X = zeros(N) if isScalar else zeros((N,)+X0.shape)
    X[0] = X0

    # Looping over an independent variable, perform Euler integration
    for i in range(N-1):
        t = bounds[0]+i*dt

        # Compute array of slopes
        X[i+1] = X[i] + diffEq(t+dt, X[i], *diffArgs)*dt
        
    return X

def RK4(X0, bounds, dt, diffEq, *diffArgs):
    """The standard 4th order Runge-Kutta integrator"""
    
    isScalar = isinstance(X0,(int,float))

    # Define integrator parameters
    N = int(round(ceil((bounds[1]-bounds[0]))/dt))+1
    a = array([1/2,1/2,1])
    b = array([1/6, 1/3, 1/3, 1/6])
    c = a

    # Intialize array of state scalars or vectors
    X = zeros(N) if isScalar else zeros((N,)+X0.shape)
    X[0] = X0

    # Looping over an independent variable, perform RK4 integration
    for i in range(N-1):
        t = bounds[0]+i*dt

        # Compute array of slopes
        ks = zeros(4) if isScalar else zeros((4,)+X0.shape)
        ks[0] = diffEq(t, X[i], *diffArgs)
        for j in range(3):
            if j==0: dXk = sum(ks[:j+1].squeeze()*a[:j+1])
            else: dXk = ks[:j+1].squeeze().T@a[:j+1]
            ks[j+1] = diffEq(t+c[j]*dt, X[i]+dXk*dt, *diffArgs)
        
        # Compute next step using slopes & parameters
        X[i+1] = X[i] + b@ks*dt

    return X

def RK45(X0, bounds, dt, diffEq, *diffArgs):
    """The Dormand-Prince Runge-Kutta integrator"""
    
    isScalar = isinstance(X0,(int,float))

    # Define integrator parameters
    N = int(round(ceil(abs(bounds[1]-bounds[0]))/abs(dt)))+1
    a = [array([1/5]), array([3/40,9/40]), array([44/45, -56/15, 32/9]), \
        array([19372/6561, -25360/2187, 64448/6571, -212/729]), \
        array([9017/3168, -355/33, 46732/5247, 49/176, -5103/18656]), \
        array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84])]
    b = array([35/384, 0, 500/1113, 125/192, -2187/6784, 11/84, 0])
    c = array([1/5, 3/10, 4/5, 8/9, 1, 1])

    # Intialize state scalar or vector
    X = zeros(N) if isScalar else zeros((N,)+X0.shape)
    X[0] = X0

    # Looping over time, perform RK4 integration
    for i in range(N-1):
        t = bounds[0]+i*dt

        # Compute array of slopes
        ks = zeros(7) if isScalar else zeros((7,)+X0.shape)
        ks[0] = diffEq(t, X[i], *diffArgs)
        for j in range(6):
            if j==0: dXk = sum(ks[:j+1].squeeze()*a[j])
            else: dXk = ks[:j+1].squeeze().T@a[j]
            ks[j+1] = diffEq(t+c[j]*dt, X[i]+dXk*dt, *diffArgs)
        
        # Compute next step using slopes & parameters
        X[i+1] = X[i] + b@ks*dt
        
    return X

In [ ]:
### jiles_atherton.py
### November 2022
### A Jiles-Atherton model for the hysteresis of a magnetic material

from numpy import array, zeros, sinh, cosh, sin, cos, exp, ceil, argsort, diff, sign
from pickle import dump, load

from .. import PI, MU0
from .integrators import *

# Define numerical integrator options
INTEGRATORS = {
    'EULER': EULER,
    'RK4': RK4,
    'RK45': RK45,
}

class JilesAtherton:
    '''
    A Jiles-Atherton model of magnetic hysteresis
    Parameters:
        alpha: Domain coupling strength
        a: Domain wall density
        Ms: Saturation magnetization of material
        k: Pinning site breaking energy
        c: Magnetization reversability
        wa: Relative weight of anisotropic contributions to magnetization
        Ka: Average anisotropy energy density
        psi: Offset angle between anisotropy easy axis & magnetizing field
    '''
    
    def __init__(self, alpha, a, Ms, k, c, wa=0, Ka=0, psi=0, dH=1, integrator='RK4'):

        # Raise exceptions for invalid input parameters
        if alpha<=0: raise ValueError("\"alpha\" (domain coupling strength) must be a positive, non-zero number")
        if a<=0: raise ValueError("\"a\" (domain wall density) must be a positive, non-zero number")
        if Ms<=0: raise ValueError("\"Ms\" (saturation magnetization) must be a positive, non-zero number")
        if k<=0: raise ValueError("\"k\" (pin-breaking energy) must be a positive, non-zero number")
        if c<=0: raise ValueError("\"c\" (magnetization reversability) must be a positive, non-zero number")
        if wa<0 or wa>1: raise ValueError("\"wa\" (anisotropy weight) must be at least zero and at most one")
        if Ka<0: raise ValueError("\"Ka\" (average anisotropy density) must be zero or a positive number")
        if psi<0 or psi>PI: raise ValueError("\"psi\" (anisotropy easy axis offset angle) must be at least zero and at most pi")
        if dH<=0: raise ValueError("\"dH\" (field step-size) must be a positive, non-zero number")
        if integrator.upper() not in INTEGRATORS.keys():
            raise ValueError("\"integrator\" must be one of {:s}".format(', '.join(INTEGRATORS.keys())))

        # Set valid input parameters
        self._alpha = alpha
        self._a = a
        self._Ms = Ms
        self._k = k
        self._c = c
        self._wa = wa
        self._Ka = Ka
        self._psi = psi
        self._dH = dH
        self._integrator= INTEGRATORS[integrator.upper()]      

        # Enforce isotropy if specified by EITHER Ka or wa parameters
        self._Ka = Ka/MU0
        if (not self._wa) or (not self._Ka): self._Ka = self._psi = self._wa = 0        
        
        # Compute the major hysteresis loop & critical points
        self._get_major()
        self._get_criticals()
                    
    # Computes the anhysteretic, isotropic magnetization
    def _Mai(self, He):
        return self._Ms*(cosh(He/self._a)/sinh(He/self._a)-self._a/He)
    
    # Computes the anhysteretic, isotropic magnetization derivative
    def _dMai(self, He):
        return self._Ms*(self._a/He**2-1/(self._a*sinh(He/self._a)**2))
    
    # Computes the anhysteretic, anisotropic magnetization
    def _Maa(self, He):
        
        # Define the integrand of the denominator
        def _denominator_integrand(theta):
            E1 = (He*cos(theta)-(self._Ka/self._Ms)*sin(self._psi-theta)**2)/self._a
            E2 = (He*cos(theta)-(self._Ka/self._Ms)*sin(self._psi+theta)**2)/self._a
            return exp((E1+E2)/2)*sin(theta)
        
        # Define the integrand of the numerator
        def _numerator_integrand(theta):
            return _denominator_integrand(theta)*cos(theta)
        
        # Compute the integrals & return the result
        return self._Ms*KRON15(_numerator_integrand, (0, PI))/KRON15(_denominator_integrand, (0, PI))
    
    # Computes the anhysteretic, anisotropic mangetization derivative
    def _dMaa(self, He):
        return (self._Maa(He+self._dH)-self._Maa(He-self._dH))/(2*self._dH)
             
    # Computes the Jiles-Atherton differential equation for magnetization (returns dM/dH)
    def _diffeq(self, H, M, dH_sign):
        
        # Compute the effective magnetic field
        He = H + self._alpha*M
        
        # Compute & add anhysteretic, isotropic magnetization contributions
        Ma = (1-self._wa)*self._Mai(He)
        dMa = (1-self._wa)*self._dMai(He)
        
        # Compute & add anfisotropic contributions (if any)
        if self._wa:
            Ma += self._wa*self._Maa(He)
            dMa += self._wa*self._dMaa(He)
            
        # Compute & return the magnetization differential
        return ((Ma-M)/(dH_sign*self._k-self._alpha*(Ma-M))+self._c*dMa)/(1+self._c)
        
    # Computes the major magnetic hysteresis curve of the model
    def _get_major(self):
        
        # Initialize the applied field & magnetization
        H = [0]
        M = [1e-6]
        
        # Increase applied field until saturation is reached
        delta_M = 1
        while (delta_M > 2.5e-5):
            
            # Increase applied field & compute the magnetization 
            H.append(H[-1] + self._dH)
            M.append(self._integrator(M[-1], (H[-2], H[-1]), self._dH, self._diffeq, 1)[1])

            # Determine the relative change in magnetization over this step
            dMdH = abs(M[-1]-M[-2])/self._dH
            delta_M = dMdH/M[-2]
            
        n_curve = int(round(ceil(2*max(H)/self._dH)))
            
        # Decrease applied field until negative saturation is reached
        H += [max(H)-self._dH*n for n in range(n_curve+1)]
        M += list(self._integrator(M[-1], (max(H), -max(H)), -self._dH, self._diffeq, -1))
        
        # Increase applied field until saturation is reached again
        H += [min(H)+self._dH*n for n in range(n_curve+1)]
        M += list(self._integrator(M[-1], (min(H), max(H)), self._dH, self._diffeq, 1))        
            
        # Assign the hysteresis variables to model class members
        self.H_major = array(H)
        self.B_major = MU0*array(M)
        
    # Computes the material coercivity & remanent field
    def _get_criticals(self):        
        
        # Separate the major loop into upper & lower sections
        upper_inds = range(int(.2*len(self.H_major)), int(.6*len(self.H_major)))
        lower_inds = range(int(.6*len(self.H_major)), int(len(self.H_major)))
        H_upper = self.H_major[upper_inds]
        H_lower = self.H_major[lower_inds]
        B_upper = self.B_major[upper_inds]
        B_lower = self.B_major[lower_inds]
        
        # Determine the remanent field
        if 0 in H_upper:
            Bru = float(B_upper[H_upper==0])
        else:
            H0 = int(diff(sign(H_upper)).nonzero()[0])
            Bru = float(B_upper[H0:H0+2].sum()/2)
        if 0 in H_lower:
            Brl = float(B_lower[H_lower==0])
        else:
            H0 = int(diff(sign(H_lower)).nonzero()[0])
            Brl = float(B_lower[H0:H0+2].sum()/2)
        self.remanence = array([Brl, Bru])
            
        # Determine the coercivity
        if 0 in B_upper:
            Hcu = float(H_upper[B_upper==0])
        else:
            B0 = int(diff(sign(B_upper)).nonzero()[0])
            Hcu = float(H_upper[B0:B0+2].sum()/2)
        if 0 in B_lower:
            Hcl = float(H_lower[B_lower==0])
        else:
            B0 = int(diff(sign(B_lower)).nonzero()[0])
            Hcl = float(H_lower[B0:B0+2].sum()/2)
        self.coercivity = array([Hcu, Hcl])
    
    # Use the model to compute hysteresis curves along a path of applied fields
    def path(self, H_path, M0):
        
        # Determine numbers of hysteresis points & initialize hysteresis variables
        N_path = (ceil(H_path.ptp(axis=1))/self._dH).round().astype('int32')+1
        H = zeros(N_path.sum())
        M = zeros(N_path.sum())
        
        # Loop over magnetizing field paths
        for p in range(len(H_path)):
            
            # Define starting & stopping field strengths for this path
            Hstart, Hstop = H_path[p]
            sgn = 1 if Hstop > Hstart else -1
            
            # Determine starting index & number of points on this path
            N0 = N_path[:p].sum()
            N = N_path[p]
            
            # Compute the hysteresis along this path
            H[N0:N0+N] = Hstart + array(range(N))*self._dH*sgn
            M[N0:N0+N] = self._integrator(M0, (Hstart, Hstop), self._dH*sgn, self._diffeq, sgn)
            M0 = M[N0+N-1]
        
        return H, MU0*M
    
    # Use the model to compute magnetic flux density at a particular field strength
    def point(self, H_point, curve='upper'):
        
        # Indices for points along each curve
        curve_indices = {
            'initial': (0, int(.2*len(self.H_major))),
            'upper': (int(.2*len(self.H_major)), int(.6*len(self.H_major))),
            'lower': (int(.6*len(self.H_major)), int(len(self.H_major)))
        }
        
        # Select points along the desired curve
        if curve.lower() in curve_indices.keys():
            indices = curve_indices[curve.lower()]
        else:
            print("WARNING: Curve choice not understood, using upper curve.")
            indices = curve_indices['upper']
        H = self.H_major[indices[0]:indices[1]]
        B = self.B_major[indices[0]:indices[1]]
        
        # Find curve point closest to prescribed point
        H_dists = abs(H - H_point)
        neighbors = argsort(H_dists)[:2]
            
        # Interpolate magnetic flux density between neighboring points
        return sum([B[pt]*H_dists[pt] for pt in neighbors])/self._dH
    
    #
    @classmethod
    def fit(cls, **fit_params):
        """
        
        Parameters:
            
        """
        return 0
    
    # Saves a Jiles-Atherton model to a file
    def save(self, path="./ja_model.pkl"):
        with open(path,'wb') as file:
            dump(self, file)
    
    # Loads a Jiles-Atherton model from a file (possibly on instantiation)
    @classmethod
    def load(cls, path="./ja_model.pkl"):
        with open(path, 'rb') as file:
            return load(file)